In [1]:
using StochasticDiffEq
using DiffEqCallbacks
using DifferentialEquations
using Plots
using LinearAlgebra
using DelimitedFiles
using DataFrames
using LinearAlgebra
using SparseArrays



In [2]:
directory_path = "/home/callum/ASF/"

"/home/callum/ASF/"

In [20]:
include(string(directory_path, "Modules/ASF_input.jl"));
include(string(directory_path, "Modules/ASF_model.jl"));

In [4]:
data_1 = ASF_Inputs.Model_Data(directory_path); #creating parameters to run model

-------------------------------------
Single Population!
The Only Population is Seeded With ASF!
-------------------------------------


In [5]:
nt = data_1.Parameters.Populations.cum_sum[end] #total number of groups and farms
nc = 5 #number of classes
eqs = 10#number of processes
n_ens = 100 #number of runs in ensemble

dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1

dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [6]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
end


regular_c (generic function with 1 method)

In [7]:
#single run with frequency inter group interections
rj = RegularJump(ASF_Model.frequency_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)

prob = DiscreteProblem(data_1.U0,data_1.Time,data_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

#sol = solve(jump_prob,SimpleTauLeaping(), dt=.1)

ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
output_f = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_ens,EnsembleThreads(),dt=.1);

In [8]:
ASF_Model.analyse_out(output_f, data_1.Parameters.Populations)

Need to Reduce Timestep
------------------------------------------------
Run Stats:
1 Populations
100 Runs
------------------------------------------------
% of groups exposed to ASF in each population:
[29.12]
------------------------------------------------
% of groups that survive in each population:
[90.9]
------------------------------------------------


In [9]:
#ensemble run with density inter group interections

rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)

prob = DiscreteProblem(data_1.U0,data_1.Time,data_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

sol = solve(jump_prob,SimpleTauLeaping(), dt=.1)

ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
output = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_ens,EnsembleThreads(),dt=.1);

In [10]:
ASF_Model.analyse_out(output, data_1.Parameters.Populations)

------------------------------------------------
Run Stats:
1 Populations
100 Runs
------------------------------------------------
% of groups exposed to ASF in each population:
[14.51]
------------------------------------------------
% of groups that survive in each population:
[95.39]
------------------------------------------------


In [ ]:
ASF_Model.save_output(output, data_1, directory_path, "NT_Frequency")

In [21]:
ASF_Model.plot_network(data_1,output)